In [1]:

import pandas as pd
import pickle

In [2]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ipsit_misra\AppData\Local\Continuum\anaconda4

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.3.0               |        py37_1001         478 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         576 KB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py37_1001 conda-forge
    branca:  0.3.1-py_0      conda-forge
    folium


folium-0.5.0         | 45 KB     |            |   0% 
folium-0.5.0         | 45 KB     | ##6        |  26% 
folium-0.5.0         | 45 KB     | ########## | 100% 

altair-2.3.0         | 478 KB    |            |   0% 
altair-2.3.0         | 478 KB    | #          |  10% 
altair-2.3.0         | 478 KB    | #####      |  50% 
altair-2.3.0         | 478 KB    | ########3  |  83% 
altair-2.3.0         | 478 KB    | #########3 |  93% 
altair-2.3.0         | 478 KB    | ########## | 100% 

vincent-0.4.4        | 28 KB     |            |   0% 
vincent-0.4.4        | 28 KB     | ########## | 100% 

branca-0.3.1         | 25 KB     |            |   0% 
branca-0.3.1         | 25 KB     | ########## | 100% 


Folium installed
Libraries imported.


In [7]:
CLIENT_ID = 'V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS'
CLIENT_SECRET = 'TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO' 
VERSION = '20190119' 
LIMIT= 40


In [50]:
address = 'Hartford, CT'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.764582 -72.6908547


In [51]:
search_query = 'school'
radius = 4000
print(search_query + ' .... OK!')

school .... OK!


In [52]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS&client_secret=TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO&ll=41.764582,-72.6908547&v=20190119&query=school&radius=4000&limit=40'

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c44f2a0dd5797605852d93a'},
 'response': {'venues': [{'id': '4c7b9511a86837049eab164d',
    'name': 'Hartford Public High School',
    'location': {'address': '55 Forest St',
     'lat': 41.764945870490145,
     'lng': -72.70094995525955,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.764945870490145,
       'lng': -72.70094995525955}],
     'distance': 839,
     'postalCode': '06105',
     'cc': 'US',
     'city': 'Hartford',
     'state': 'CT',
     'country': 'United States',
     'formattedAddress': ['55 Forest St',
      'Hartford, CT 06105',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'shortName': 'High School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1548022432',
    'hasPerk': False},
   {'id': '4bdb23ff63c5c

In [54]:
venues

[{'id': '4e568484887784077062a95d',
  'name': 'Italian Gourmet',
  'location': {'address': '15 Hartford Ave',
   'crossStreet': 'Main Street',
   'lat': 41.7126237154582,
   'lng': -72.72542268461268,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.7126237154582,
     'lng': -72.72542268461268}],
   'distance': 1343,
   'postalCode': '06111',
   'cc': 'US',
   'city': 'Newington',
   'state': 'CT',
   'country': 'United States',
   'formattedAddress': ['15 Hartford Ave (Main Street)',
    'Newington, CT 06111',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d146941735',
    'name': 'Deli / Bodega',
    'pluralName': 'Delis / Bodegas',
    'shortName': 'Deli / Bodega',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '272914',
   'url': 'https://www.grubhub.com/restaurant/italian-gourmet-15-hartford-ave-newington/272914?affiliate=1131&utm_source=foursquare-affiliate-ne

In [55]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4c7b9511a86837049eab164d,55 Forest St,US,Hartford,United States,NaN,839,"[55 Forest St, Hartford, CT 06105, United States]","[{'label': 'display', 'lat': 41.76494587049014...",41.764946,-72.700950,06105,CT,Hartford Public High School,v-1548022432,NaN
1,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",False,4bdb23ff63c5c9b60b882668,5 Cone St,US,Hartford,United States,Whitney Street,1839,"[5 Cone St (Whitney Street), Hartford, CT 0610...","[{'label': 'display', 'lat': 41.76854024561700...",41.768540,-72.712369,06105,CT,Noah Webster MicroSociety Magnet School,v-1548022432,NaN
2,"[{'id': '4bf58dd8d48988d1a6941735', 'name': 'L...",False,4bba0edc1261d13ae127ea98,65 Elizabeth St,US,Hartford,United States,NaN,1485,"[65 Elizabeth St, Hartford, CT 06105, United S...","[{'label': 'display', 'lat': 41.77176636029461...",41.771766,-72.705932,06105,CT,University of Connecticut School of Law,v-1548022432,NaN
3,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",False,4c992442292a6dcbd7c2c376,400 Wethersfield Ave,US,Hartford,United States,NaN,2426,"[400 Wethersfield Ave, Hartford, CT 06114, Uni...","[{'label': 'display', 'lat': 41.74742571690222...",41.747426,-72.672828,06114,CT,Morgan Gardner Bulkeley High School,v-1548022432,NaN
4,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,4f8c2ce6e4b0bae8998a1329,NaN,US,Hartford,United States,NaN,1297,"[Hartford, CT 06103, United States]","[{'label': 'display', 'lat': 41.76802664112197...",41.768027,-72.675929,06103,CT,University of Saint Joseph School of Pharmacy,v-1548022432,34285654


In [56]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hartford Public High School,High School,55 Forest St,US,Hartford,United States,NaN,839,"[55 Forest St, Hartford, CT 06105, United States]","[{'label': 'display', 'lat': 41.76494587049014...",41.764946,-72.700950,06105,CT,4c7b9511a86837049eab164d
1,Noah Webster MicroSociety Magnet School,School,5 Cone St,US,Hartford,United States,Whitney Street,1839,"[5 Cone St (Whitney Street), Hartford, CT 0610...","[{'label': 'display', 'lat': 41.76854024561700...",41.768540,-72.712369,06105,CT,4bdb23ff63c5c9b60b882668
2,University of Connecticut School of Law,Law School,65 Elizabeth St,US,Hartford,United States,NaN,1485,"[65 Elizabeth St, Hartford, CT 06105, United S...","[{'label': 'display', 'lat': 41.77176636029461...",41.771766,-72.705932,06105,CT,4bba0edc1261d13ae127ea98
3,Morgan Gardner Bulkeley High School,High School,400 Wethersfield Ave,US,Hartford,United States,NaN,2426,"[400 Wethersfield Ave, Hartford, CT 06114, Uni...","[{'label': 'display', 'lat': 41.74742571690222...",41.747426,-72.672828,06114,CT,4c992442292a6dcbd7c2c376
4,University of Saint Joseph School of Pharmacy,College Academic Building,NaN,US,Hartford,United States,NaN,1297,"[Hartford, CT 06103, United States]","[{'label': 'display', 'lat': 41.76802664112197...",41.768027,-72.675929,06103,CT,4f8c2ce6e4b0bae8998a1329
5,McDonough Expeditionary Learning School,School,111 Hillside Ave,US,Hartford,United States,NaN,1788,"[111 Hillside Ave, Hartford, CT 06106, United ...","[{'label': 'display', 'lat': 41.74949740584814...",41.749497,-72.698258,06106,CT,4d860d3802eb5481bbb14ff5
6,The Hartt School,College Arts Building,"200 Bloomfield Avenue, Fuller Building",US,West Hartford,United States,NaN,4154,"[200 Bloomfield Avenue, Fuller Building, West ...","[{'label': 'display', 'lat': 41.79751331640804...",41.797513,-72.714394,06119,CT,4ba81da3f964a520abcd39e3
7,Burns School,School,NaN,US,Hartford,United States,NaN,350,"[Hartford, CT 06106, United States]","[{'label': 'display', 'lat': 41.76145810039826...",41.761458,-72.691390,06106,CT,4ec54d9ebe7bd3e1852b816a
8,William T. Sedgwick Middle School,Middle School,128 Sedgwick Rd,US,West Hartford,United States,NaN,5554,"[128 Sedgwick Rd, West Hartford, CT 06107, Uni...","[{'label': 'display', 'lat': 41.75177110302203...",41.751771,-72.755499,06107,CT,4b9295fdf964a520fe0634e3
9,The Grace S Webb School,School,200 Retreat Ave.,US,Hartford,United States,at Vernon St.,1954,"[200 Retreat Ave. (at Vernon St.), Hartford, C...","[{'label': 'display', 'lat': 41.74825649366399...",41.748256,-72.682204,06114,CT,4fdb2e30e4b061c120d15b4e


In [57]:
dataframe_filtered.name

0                           Hartford Public High School
1               Noah Webster MicroSociety Magnet School
2               University of Connecticut School of Law
3                   Morgan Gardner Bulkeley High School
4         University of Saint Joseph School of Pharmacy
5               McDonough Expeditionary Learning School
6                                      The Hartt School
7                                          Burns School
8                     William T. Sedgwick Middle School
9                               The Grace S Webb School
10                Annie Fisher Montessori Magnet School
11                              Opportunity High School
12    New Testament Baptist Church & School (inside ...
13                      Maria Sanchez Elementary School
14                             Morley Elementary School
15                                        Sawyer School
16                           Capital Prep Magnet School
17                                           Old

In [74]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=3) 

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hartford',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [76]:
venue_id = '4c7b9511a86837049eab164d' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c7b9511a86837049eab164d?client_id=V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS&client_secret=TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO&v=20190119'

In [77]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '4c7b9511a86837049eab164d',
 'name': 'Hartford Public High School',
 'contact': {},
 'location': {'address': '55 Forest St',
  'lat': 41.764945870490145,
  'lng': -72.70094995525955,
  'labeledLatLngs': [{'label': 'display',
    'lat': 41.764945870490145,
    'lng': -72.70094995525955}],
  'postalCode': '06105',
  'cc': 'US',
  'city': 'Hartford',
  'state': 'CT',
  'country': 'United States',
  'formattedAddress': ['55 Forest St', 'Hartford, CT 06105', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/hartford-public-high-school/4c7b9511a86837049eab164d',
 'categories': [{'id': '4bf58dd8d48988d13d941735',
   'name': 'High School',
   'pluralName': 'High Schools',
   'shortName': 'High School',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d13b941735',
   'name': 'School',
   'pluralName': 'Schools',
   'shortName': 'School',
   'icon': {'prefix': 'https://ss3.4sqi.

In [78]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [86]:
search_query = 'school'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS&client_secret=TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO&ll=41.764582,-72.6908547&v=20190119&radius=4000&limit=40'

In [87]:
import requests

In [88]:
results = requests.get(url).json()
'There are {} around school.'.format(len(results['response']['groups'][0]['items']))

'There are 40 around school.'

In [89]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '597b1d80dec1d60e99844f21',
  'name': 'Story and Soil',
  'location': {'address': '387 Capitol Ave',
   'lat': 41.76385726287704,
   'lng': -72.68851654518593,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.76385726287704,
     'lng': -72.68851654518593}],
   'distance': 210,
   'postalCode': '06106',
   'cc': 'US',
   'city': 'Hartford',
   'state': 'CT',
   'country': 'United States',
   'formattedAddress': ['387 Capitol Ave',
    'Hartford, CT 06106',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-597b1d80dec

In [90]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Story and Soil,Coffee Shop,387 Capitol Ave,US,Hartford,United States,NaN,210,"[387 Capitol Ave, Hartford, CT 06106, United S...","[{'label': 'display', 'lat': 41.76385726287704...",41.763857,-72.688517,NaN,06106,CT,597b1d80dec1d60e99844f21
1,Little River Restoratives,Speakeasy,405 Capitol Ave,US,Hartford,United States,Babcock St,179,"[405 Capitol Ave (Babcock St), Hartford, CT 06...","[{'label': 'display', 'lat': 41.76379360903065...",41.763794,-72.688972,NaN,06106,CT,5642673e498ee124a7098a40
2,Bushnell Center for the Performing Arts,Performing Arts Venue,166 Capitol Ave,US,Hartford,United States,at Trinity St.,842,"[166 Capitol Ave (at Trinity St.), Hartford, C...","[{'label': 'display', 'lat': 41.76291575371237...",41.762916,-72.680961,NaN,06106,CT,4b6e0e25f964a520ffa62ce3
3,The Mark Twain House & Museum,History Museum,351 Farmington Ave,US,Hartford,United States,NaN,879,"[351 Farmington Ave, Hartford, CT 06105, Unite...","[{'label': 'display', 'lat': 41.76644019220782...",41.766440,-72.701145,NaN,06105,CT,4b799844f964a52026042fe3
4,Bushnell Park,Park,Jewell Street,US,Hartford,United States,Trinity Street,993,"[Jewell Street (Trinity Street), Hartford, CT ...","[{'label': 'display', 'lat': 41.76519607484078...",41.765196,-72.678913,NaN,06103,CT,4b97cbfbf964a5202a1635e3
5,Firebox Restaurant,New American Restaurant,539 Broad St,US,Hartford,United States,btw Russ & Capitol,412,"[539 Broad St (btw Russ & Capitol), Hartford, ...","[{'label': 'display', 'lat': 41.76214395021058...",41.762144,-72.687107,NaN,06106,CT,4b09dd6ff964a520e31e23e3
6,Salute,Italian Restaurant,100 Trumbull St,US,Hartford,United States,NaN,1213,"[100 Trumbull St, Hartford, CT 06103, United S...","[{'label': 'display', 'lat': 41.76523097667638...",41.765231,-72.676265,NaN,06103,CT,4b8727b4f964a520a5b331e3
7,Real Art Ways - Cinema,Art Gallery,56 Arbor St,US,Hartford,United States,off Capitol Avenue,1142,"[56 Arbor St (off Capitol Avenue), Hartford, C...","[{'label': 'display', 'lat': 41.75939109308119...",41.759391,-72.702726,NaN,06106,CT,4b1479d1f964a5209ca323e3
8,Max Downtown,American Restaurant,185 Asylum St,US,Hartford,United States,NaN,1215,"[185 Asylum St, Hartford, CT 06103, United Sta...","[{'label': 'display', 'lat': 41.76714093568001...",41.767141,-72.676628,NaN,06103,CT,4b662d24f964a52096172be3
9,Blackeyed Sally's,BBQ Joint,350 Asylum St,US,Hartford,United States,NaN,1000,"[350 Asylum St, Hartford, CT 06103, United Sta...","[{'label': 'display', 'lat': 41.76766123274433...",41.767661,-72.679532,NaN,06103,CT,4b05ff3cf964a52028e722e3


In [91]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map